## Scrape https://magic.wizards.com/en/news/archive and collect the articles

Imports

In [77]:
import datetime
import ipywidgets as widgets
import urllib
import requests
from bs4 import BeautifulSoup

Set search URL (page one is still hardcoded) nd the search date (current date as default)

In [78]:
from_date = datetime.date.today()
search_url='https://magic.wizards.com/en/news/archive?search&page=1&category=all&author=all&order=newest'

Make the date easy changeable

In [79]:
widgets.DatePicker(
    description = 'Start Date',
    value = from_date
)

DatePicker(value=datetime.date(2023, 2, 7), description='Start Date', step=1)

Query the search page to retrieve the links for the articles

In [80]:
search_response = requests.get(search_url)

In [81]:
if (search_response.status_code != 200):
    raise requests.HTTPError(search_response)


Parse HTML

In [82]:
page_result = search_response.text
soup = BeautifulSoup(page_result, 'html.parser')
page_list = []

Extract the links to the articles (defined by css class css-9f4rq)

In [83]:
entry_list = soup.find_all(class_="css-9f4rq")
for entry in entry_list:
    parent = entry.findParent()
    page_list.append('https://magic.wizards.com'+parent.get('href'))

Query the article links and save the HTML as file for later use

In [84]:
for page in page_list:
    response = requests.get(page)
    if (response.status_code != 200):
        raise requests.HTTPError(response)
    file_name = './data/raw_html/' + page.split('/')[-1]
    # print(file_name)
    with open(file_name, mode='w') as f:
        f.write(response.text)
    